In [1]:
from keras.layers import *
from keras.models import Model

In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
folder_path = "C:/Users/irbag/Downloads/archive (6)/UTKFace"

In [4]:
age = []
gender = []
img_path = []

for file in os.listdir(folder_path):
    try:
        parts = file.split('_')
        
        # Append the first part (age) to the age list
        age.append(int(parts[0]))
        
        # Append the second part (gender) to the gender list
        gender.append(int(parts[1]))
        
        # Append the full filename to the img_path list
        img_path.append(file)
        
    except Exception as e:
        # This will skip any files that don't match the format (like .DS_Store or README)
        print(f"Skipping file: {file}, Error: {e}")

In [5]:
len(age)

23708

In [6]:
df = pd.DataFrame({'age':age, 'gender':gender, 'img':img_path})

In [7]:
df.shape

(23708, 3)

In [8]:
df.head()

,age,gender,img
0,100,0,100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,100_1_0_20170112213303693.jpg.chip.jpg


In [9]:
train_df = df.sample(frac=1, random_state=0).iloc[:20000]
test_df = df.sample(frac=1, random_state=0).iloc[20000:]

In [10]:
train_df.shape

(20000, 3)

In [11]:
test_df.shape

(3708, 3)

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
# 1. Define your batch size (default is 32)
batch_size = 32 

# 2. Re-create your generators, adding the batch_size explicitly
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output',
                                                    batch_size=batch_size) # <-- Add this

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  directory=folder_path,
                                                  x_col='img',
                                                  y_col=['age','gender'],
                                                  target_size=(200,200),
                                                  class_mode='multi_output',
                                                  batch_size=batch_size) # <-- Add this


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [14]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [15]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [16]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [17]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [18]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [22]:
# 3. Calculate the steps per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(test_df) // batch_size


# 4. Create the wrapper function to convert list -> tuple
def tuple_wrapper(generator):
    while True:
        x, y = next(generator)
        yield x, tuple(y) # <-- This is the fix

# 5. Wrap your generators
train_gen_wrapped = tuple_wrapper(train_generator)
test_gen_wrapped = tuple_wrapper(test_generator)

# 6. Fit the model using the wrapped generators and steps
history = model.fit(
    train_gen_wrapped,
    epochs=1,
    validation_data=test_gen_wrapped,
    steps_per_epoch=steps_per_epoch,       # <-- Required
    validation_steps=validation_steps      # <-- Required
)

625/625 ━━━━━━━━━━━━━━━━━━━━ 735s 1s/step - age_loss: 15.1649 - age_mae: 15.1649 - gender_accuracy: 0.5153 - gender_loss: 0.7228 - loss: 86.7188 - val_age_loss: 15.2431 - val_age_mae: 15.2431 - val_gender_accuracy: 0.5163 - val_gender_loss: 0.6930 - val_loss: 83.8524


In [1]:
# Running for More Epochs will Increase the Accuracy